# The recursive ssk kernel using Def. 2 and the efficient calculation approach.


In [1]:
## Imports

import fetch_data as fd
import numpy as np
import itertools as it
import tqdm
from math import sqrt

In [270]:
## Methods

def k_prime(s,t,n,l):

    #-------------------------------------------------------------------------------------#
    # Basically what's happening here is that we are succesively looping through both of  #
    # the strings and updating the kernel matrix accordingly while refering to previously #
    # computed values. This will give us time complexity O(n|s||t|) in the end.           #
    #-------------------------------------------------------------------------------------#
    
    
    #Variables:
    #
    #s is a string
    #t is a string
    #n is the length of the substring
    #l is the lambda value
    #kp is refering to k'
    #kpp is refering to k'' 
    
    #start by creating the empty matrices.
    kp = np.zeros([n,len(s)+1,len(t)+1]);
    kpp = np.zeros([n,len(s)+1,len(t)+1]);
    
    #initialize
    kp[0][:][:] = 1;
    
    for i in range(1,n):
        for j in range(i,len(s)):
            for k in range(i,len(t)+1):
                #check the base cases
                #if true, no need to calculate.
                ##if(np.min([j,k])<i):
                 #   kp[i][j][k]=0;
                 #   continue;

                #check whether 'x occurs in u' as described in the paper
                if(s[j-1]!=t[k-1]):
                    kpp[i][j][k]=l*kpp[i][j][k-1];
                #if not, do the other calcs.
                else:
                    kpp[i][j][k]=l*(kpp[i][j][k-1]+l*kp[i-1][j-1][k-1]);
                
                #finally calculate kp
                kp[i][j][k-1]=l*kp[i][j-1][k-1]+kpp[i][j][k-1];
                
    return kp;


def k(s,t,n,l,kp):
    
    #--------------------------------------------------#
    # This takes in an already computed k_prime kernel #
    # and calculates the overall kernel as per the     #
    # paper. The last part of Def. 2                   #
    #--------------------------------------------------#

    #Variables:
    #
    #s is a string
    #t is a string
    #n is the length of the substring
    #l is the lambda value
    #kp is refering to k'
    #ksum is refering to the kernel value.
    
    ksum = 0;
    
    #Loop over all values in the computed k_prime matrix and 
    #pick out the values where x = j, as mentioned in the paper.
    #
    #There is no recursion necessary here since we already did it
    #when computing k_prime, the last 'layer' of k_prime
    #contains all the necessary values.
    
    for i in range(kp.shape[1]-1):
        for j in range(kp.shape[2]-1):
            if(s[i]==t[j]):
                ksum += kp[n-1][i][j];
                
    return l**2*ksum;

def get_normed_kernel_values(s,t,n,l):
    
    #------------------------------------------------------#
    # This returns the normalized values for the kernel    # 
    # using the normalization mentioned in the paper.      #
    # s is a string.                                       #
    # t is a string.                                       #
    # n is the substring length                            #           
    # l is the lambda value, the 'weight'                  #
    #------------------------------------------------------#
    
    kstP = k_prime(s,t,n,l);
    kssP = k_prime(s,s,n,l);
    kttP = k_prime(t,t,n,l);

    kst = k(s,t,n,l,kstP)
    kss = k(s,s,n,l,kssP);
    ktt = k(t,t,n,l,kttP);
    
    return(kst/sqrt(kss*ktt))

## Below are some testing cases. The same as for the naive implementation just for sanity checks.

In [269]:
testData = ["science is organized knowledge","wisdom is organized life"]
#testData = ["scat","scat"]
#testData = ["cat","car","bat","bar"]

l=0.5;


n=2;
print("K_2: " + repr(get_normed_kernel_values(testData[0],testData[1],n,l)))


n=3;
print("K_3: " + repr(get_normed_kernel_values(testData[0],testData[1],n,l)))


n=4;
print("K_4: " + repr(get_normed_kernel_values(testData[0],testData[1],n,l)))


n=5;
print("K_5: " + repr(get_normed_kernel_values(testData[0],testData[1],n,l)))

n=6;
print("K_6: " + repr(get_normed_kernel_values(testData[0],testData[1],n,l)))

K_2: 0.57981369829289153
K_3: 0.47845478625579563
K_4: 0.43887050844308401
K_5: 0.40574516353732781
K_6: 0.36915601201963888
